<a href="https://colab.research.google.com/github/Hung6884/test/blob/main/VN_Spell_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade datasets
!rm -rf ~/.cache/huggingface/datasets
!pip install huggingface_hub
!pip install evaluate
!pip install transformers
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [2]:
import warnings

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer

warnings.filterwarnings("ignore")

Config

In [3]:
MODEL_NAME = "vinai/bartpho-syllable"
MAX_LENGTH = 256

Prepare Dataset

In [4]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


In [5]:
dataset = load_dataset("bmd1905/vi-error-correction-2.0")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vi.train.csv:   0%|          | 0.00/911M [00:00<?, ?B/s]

vi.test.csv:   0%|          | 0.00/31.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2628492 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/99996 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 2628492
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 99996
    })
})

In [7]:
# reduce dataset for testing
dataset["train"] = dataset["train"].select(range(50_000))
dataset["test"] = dataset["test"].select(range(5_000))

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 5000
    })
})

In [9]:
dataset["train"][-23829]

{'input': 'Tôi học hỏi và ảh hưởng ở chị trên sân khấu và ở ngoài đời là tấm gương về sự yêu thương và vị tha.',
 'output': 'Tôi học hỏi và ảnh hưởng ở chị trên sân khấu và ở ngoài đời là tấm gương về sự yêu thương và vị tha.'}

In [10]:
# Example
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

input = dataset["train"][-1]["input"]
output = dataset["train"][-1]["output"]

# Use tokenizer for inputs
inputs = tokenizer(input, text_target=output, max_length=MAX_LENGTH, truncation=True)

inputs, len(inputs["input_ids"])

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

({'input_ids': [0, 710, 51, 341, 16, 2173, 32, 2583, 97, 90, 6900, 1028, 758, 4, 533, 127, 22, 88, 4, 54, 776, 258, 182, 64, 22, 317, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 710, 51, 341, 16, 169, 32, 2583, 97, 90, 22, 758, 4, 533, 127, 22, 88, 4, 54, 776, 258, 182, 64, 22, 317, 5, 2]},
 28)

Tokenize dataset

In [11]:
def preprocess_function(examples):
    # Tokenize the text and apply truncation
    return tokenizer(
        examples["input"],
        text_target=examples["output"],
        max_length=MAX_LENGTH,
        truncation=True,
    )


# Apply tokenization in a batched manner for efficiency
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Model

In [14]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

Metric

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
import evaluate

metric = evaluate.load("sacrebleu")

In [17]:
predictions = [
    "Nếu làm được như vậy thì chắc chắn sẽ không còn trường nào tùy tiện thu tiền cao, gây sự lo lắng của phụ huynh và ai không có tiền thì không cần đóng."
]
references = [
    [
        "Nếu làm được như vậy thì chắc chắn sẽ không còn trường nào tùy tiện thu tiền cao, gây sự lo lắng của phụ huynh và ai không có tiền thì không cần đóng."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 100.00000000000004,
 'counts': [36, 35, 34, 33],
 'totals': [36, 35, 34, 33],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 36,
 'ref_len': 36}

In [18]:
predictions = [
    "Nếu làm được như vậy thì chắc chắn sẽ khôTng còn trường nà tùy tiện tu tiềncaogây sự lo hắng của phụ huynh và ai khÔng có tiền thì kông cần dong"
]
references = [
    [
        "Nếu làm được như vậy thì chắc chắn sẽ không còn trường nào tùy tiện thu tiền cao, gây sự lo lắng của phụ huynh và ai không có tiền thì không cần đóng."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 39.85576664202889,
 'counts': [24, 17, 11, 8],
 'totals': [32, 31, 30, 29],
 'precisions': [75.0,
  54.83870967741935,
  36.666666666666664,
  27.586206896551722],
 'bp': 0.8824969025845955,
 'sys_len': 32,
 'ref_len': 36}

In [19]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"sacrebleu": result["score"]}

Fine-tuning Model

In [20]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    do_train=True,
    do_eval=True,
    output_dir="Jaye29dphy/vietnamese-correction",
    num_train_epochs=10,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4 * 4,
    gradient_accumulation_steps=2,
    eval_strategy="steps",
    eval_steps=20_000,
    save_strategy="steps",
    logging_steps=20_000,
    save_total_limit=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [21]:
import torch
torch.cuda.empty_cache()

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jaye29dphy (jaye29dphy-github) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 1.1453126668930054,
 'eval_model_preparation_time': 0.006,
 'eval_sacrebleu': 22.84666906893919,
 'eval_runtime': 183.8404,
 'eval_samples_per_second': 27.198,
 'eval_steps_per_second': 1.703}

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub(tags="text2text-generation", commit_message="Training complete")

KeyboardInterrupt: 

Inference

In [ ]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="Jaye29dphy/vietnamese-correction")

In [ ]:
MAX_LENGTH = 512

# Define the text samples
texts = [
    "côn viec kin doanh thì rất kho khan nên toi quyết dinh chuyển sang nghề khac  ",
    "toi dang là sinh diên nam hai ở truong đạ hoc khoa jọc tự nhiên , trogn năm ke tiep toi sẽ chọn chuyen nganh về trí tue nhana tạo",
    "Tôi  đang học AI ở trun tam AI viet nam  ",
    "Nhưng sức huỷ divt của cơn bão mitch vẫn chưa thấm vào đâu lsovớithảm hoạ tại Bangladesh ăm 1970 ",
    "Lần này anh Phươngqyết xếp hàng mua bằng được 1 chiếc",
    "một số chuyen gia tài chính ngâSn hànG của Việt Nam cũng chung quan điểmnày",
    "Cac so liệu cho thay ngươi dân viet nam đang sống trong 1 cuôc sóng không duojc nhu mong đọi",
    "Nefn kinh té thé giới đang đúng trươc nguyen co của mọt cuoc suy thoai",
    "Khong phai tất ca nhưng gi chung ta thấy dideu là sụ that",
    "chinh phủ luôn cố găng het suc để naggna cao chat luong nền giáo duc =cua nuoc nhà",
    "nèn kinh te thé giới đang đứng trươc nguy co của mọt cuoc suy thoai",
    "kinh tế viet nam dang dứng truoc 1 thoi ky đổi mơi chưa tung có tienf lệ trong lịch sử",
]

# Batch prediction
predictions = corrector(texts, max_length=MAX_LENGTH)

# Print predictions
for text, pred in zip(texts, predictions):
    print("- " + pred["generated_text"])